# Settings

In [29]:
IN_FILENAME = "../data/processed/calculated.xlsx"
OUT_FILENAME = "../data/processed/filtered.xlsx"

SEPARATOR = "\n"

### adjustments

In [30]:
import re

fmt = re.search("[.][a-z]+$", OUT_FILENAME)
if fmt == None: raise Exception("have to choose a format (xlsx | csv)")
fmt = fmt.group()[1:]

# Prevs

### Imports

In [31]:
import pandas as pd
import numpy as np
from collections import namedtuple
import json

### constants imports

In [32]:
with open("../constants/columns.json", "r") as file: columns, years = json.load(file).values()

columns = namedtuple('Columns', ' '.join(columns.keys()))(**columns)
years = namedtuple('Years', re.sub(r'[\[\],\']','', str([*map(lambda y: f'y{y}', years)])))(*years)

### tools

In [33]:
YYYY = lambda txt, year : txt.replace('[YYYY]', str(year))

# Main

In [34]:
filter_columns = [

   *[YYYY(columns.TD_div_TA_YYYY, year) for year in years],
   *[YYYY(columns.WC_div_TA_YYYY, year) for year in years],
   *[YYYY(columns.EBITDA_div_TA_YYYY, year) for year in years],
   *[YYYY(columns.lnTA_YYYY, year) for year in years],

   #*[YYYY(columns.z_zone_green_YYYY, year) for year in years],
   #*[YYYY(columns.z_zone_grey_YYYY, year) for year in years],
   #*[YYYY(columns.z_zone_red_YYYY, year) for year in years],
   *[YYYY(columns.z_cat_YYYY, year) for year in years],
   
   columns.isFamiliar
]

In [35]:
data = pd.read_excel(IN_FILENAME)

In [36]:
df = data[filter_columns]

In [37]:
df.head()

,TD/TA 2021,TD/TA 2022,TD/TA 2023,WC/TA 2021,WC/TA 2022,WC/TA 2023,EBITDA/TA 2021,EBITDA/TA 2022,EBITDA/TA 2023,ln(TA) 2021,ln(TA) 2022,ln(TA) 2023,Z Cat 2021,Z Cat 2022,Z Cat 2023,isFamiliar
0,0.563004,0.565019,0.574824,0.039019,0.031774,0.041247,0.050417,0.019354,0.059132,19.051693,19.030923,19.070886,1,1,1,-1
1,0.619957,0.625639,0.616133,0.231849,0.191468,0.238469,0.043954,0.030919,0.052901,19.686780,19.710178,19.721963,2,2,2,0
2,0.351584,0.396238,0.417644,0.243964,0.193162,0.176384,0.022541,0.017641,-0.008590,19.604140,19.642154,19.650698,1,1,2,-1
3,0.239194,0.119545,0.121384,0.290527,0.142736,0.080459,0.096372,0.094655,0.091838,17.311397,19.714089,19.786924,1,0,0,1
4,0.170529,0.160455,0.211733,0.413140,0.209358,0.391818,0.081600,0.077455,0.129007,19.453267,19.957565,19.514163,0,0,1,0


In [38]:
match fmt:
    case "xlsx": df.to_excel(OUT_FILENAME, index=False)
    case "csv": df.to_csv(OUT_FILENAME, index=False)
    case _: raise Exception("Stop touching things!!")